In [1]:
# Import Libraries ที่ต้องใช้
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D

# เชื่อมต่อ Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- โหลดข้อมูล ---
data_path = '/content/drive/MyDrive/Colab Notebooks/Thai_Quant_AI_Project/02_data/set50_processed_data.csv'
df = pd.read_csv(data_path, index_col='Date', parse_dates=True)

print("โหลดข้อมูลสำเร็จ!")

Mounted at /content/drive
โหลดข้อมูลสำเร็จ!


In [2]:
# --- 1. ปรับสเกลข้อมูล ---
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

# --- 2. แบ่งข้อมูล Train/Test ---
training_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[0:training_size, :]
test_data = scaled_data[training_size - 60: , :]

# --- 3. สร้างชุดข้อมูลแบบหน้าต่าง (Windowing) ---
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), :]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 3])
    return np.array(dataX), np.array(dataY)

time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("เตรียมข้อมูลสำเร็จ!")
print("Shape ของ X_train:", X_train.shape)
print("Shape ของ y_train:", y_train.shape)

เตรียมข้อมูลสำเร็จ!
Shape ของ X_train: (1852, 60, 16)
Shape ของ y_train: (1852,)


In [3]:
# --- สร้างฟังก์ชันสำหรับ Transformer Block ---
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # --- Attention Block ---
    # Multi-Head Attention: มองข้อมูลจากหลายๆ มุม
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + inputs) # Skip Connection

    # --- Feed Forward Block ---
    # ส่วนประมวลผลหลังจาก Attention
    ffn = tf.keras.Sequential([
        Dense(ff_dim, activation="relu"),
        Dense(inputs.shape[-1])
    ])
    x = ffn(x)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + x) # Skip Connection
    return x

# --- สร้างฟังก์ชันสำหรับประกอบโมเดลทั้งหมด ---
def build_transformer_model(
    input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0
):
    inputs = Input(shape=input_shape)
    x = inputs

    # --- Positional Encoding ---
    # Transformer ไม่รู้ลำดับข้อมูล เราต้องเพิ่มข้อมูลตำแหน่งเข้าไป
    positions = tf.range(start=0, limit=input_shape[0], delta=1)
    pos_embedding = tf.keras.layers.Embedding(input_dim=input_shape[0], output_dim=input_shape[1])(positions)
    x = x + pos_embedding

    # --- Stacking Transformer Blocks ---
    # สร้าง Transformer Block ซ้อนกันหลายๆ ชั้นเพื่อให้เรียนรู้ได้ลึกขึ้น
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # --- Output Block ---
    # รวมผลลัพธ์จากทุก Timestep ให้เป็นหนึ่งเดียว
    x = GlobalAveragePooling1D(data_format="channels_first")(x)

    # MLP (Multi-Layer Perceptron) เพื่อทำนายผลสุดท้าย
    for dim in mlp_units:
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)

    outputs = Dense(1)(x) # Output สุดท้ายคือราคาที่ทำนาย
    return Model(inputs, outputs)

# --- กำหนดค่า Hyperparameters และสร้างโมเดล ---
input_shape = X_train.shape[1:]

model = build_transformer_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

# Compile โมเดล
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 16)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 16)    │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 16)    │     68,624 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 60, 16)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 16)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 16)    │         32 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 60, 16)    │        148 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 16)    │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 16)    │          0 │ dropout_2[0][0],  │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 16)    │     68,624 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 16)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 16)    │          0 │ dropout_4[0][0],  │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 60, 16)    │        148 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 60, 16)    │          0 │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 60, 16)    │          0 │ dropout_5[0][0],  │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 16)    │         32 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 283,281 (1.08 MB)

 Trainable params: 283,281 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# เริ่มการฝึกสอนโมเดล
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=64,
    verbose=1
)

# --- บันทึกโมเดลที่เทรนเสร็จแล้ว ---
model_path = '/content/drive/MyDrive/Colab Notebooks/Thai_Quant_AI_Project/03_models/transformer_upgrade_model.h5'
model.save(model_path)
print(f"โมเดลถูกบันทึกเรียบร้อยที่: {model_path}")

Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - loss: 0.3612 - val_loss: 0.2215
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 0.0466 - val_loss: 0.1155
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 0.0327 - val_loss: 0.1079
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 0.0348 - val_loss: 0.1102
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 0.0346 - val_loss: 0.1024
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 0.0333 - val_loss: 0.1027
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 0.0341 - val_loss: 0.1074
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 0.0341 - val_loss: 0.1197
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 0.0348 - val_loss: 0.1144
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 0.0330 - val_loss: 0.1094
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 0.0313 - val_loss: 0.1174
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - loss: 0.0321 - val_lo

โมเดลถูกบันทึกเรียบร้อยที่: /content/drive/MyDrive/Colab Notebooks/Thai_Quant_AI_Project/03_models/transformer_upgrade_model.h5
